In [87]:
from cmj.loa import models
from cmj.utils import Manutencao, run_sql
from django.conf import settings
from decimal import Decimal
import fitz
import re
m = Manutencao()
m.desativa_auto_now()
m.desativa_signals()

In [88]:
d = AnexoNormaJuridica.objects.get(pk=74)
settings.SITE_URL+d.anexo_arquivo.url

'https://www.jatai.go.leg.br/api/norma/anexonormajuridica/74/anexo_arquivo/'

In [89]:
#dfile = fitz.open(d.anexo_arquivo.path)
dfile = fitz.open('/home/leandro/Downloads/anexo_norma.pdf')

In [90]:
def load_pages_with(page_mask=''):
    rtext = ''
    for np in range(len(dfile)):
        page = dfile.load_page(np)
        text = page.get_text("text")

        if page_mask not in text:
            continue

        rtext += text
    while '  ' in rtext:
        rtext = rtext.replace('  ', ' ')
    while ' \n' in rtext:
        rtext = rtext.replace(' \n', '\n')
    while '\n\n' in rtext:
        rtext = rtext.replace('\n\n', '\n')
    return rtext

In [91]:
text_global = load_pages_with(page_mask='Quadro de Detalhamento da Despesa \nÓRGÃO \n')

In [96]:
len(text_global), text_global[34670:35720]

(72936,
 'PERFURAÇÃO DE 2 POÇOS ARTESIANOS\n27.122.2739.1.216\n4.4.90.51.00\n165.000,00\n165.000,00\nMANUTENÇÃO DA SECRETARIA DE ESPORTE\n27.122.2739.2.102\n3.1.90.04.00\n1.000,00\n3.1.90.11.00\n1.073.382,89\n3.1.90.91.00\n800,00\n3.3.90.08.00\n300,00\n3.3.90.14.00\n7.950,00\n3.3.90.30.00\n202.657,51\n3.3.90.32.00\n10.000,00\n3.3.90.34.00\n3.000,00\n3.3.90.36.00\n8.000,00\n3.3.90.39.00\n37.290,84\n3.3.90.40.00\n350,00\n3.3.90.47.00\n300,00\n3.3.90.91.00\n500,00\n3.3.90.92.00\n250,00\n3.3.90.93.00\n529,80\n1.346.511,04\nAMPLIAÇÃO/REFORMA DO GINÁSIO JK\n27.812.2739.1.145\n4.4.90.51.00\n1.000.000,00\n1.000.000,00\nCONSTRUÇÃO DO COMPLEXO ESPORTIVO - JK\n27.812.2739.1.147\n4.4.90.51.00\n750.000,00\n750.000,00\nIMPLANT./MODERNIZAÇÃO CAMPOS DE FUTEBOL\n27.812.2739.1.148\n4.4.90.51.00\n377.500,00\n4.4.90.52.00\n200.000,00\n577.500,00\nAMPLIAÇÃO E REFORMA DO ESTÁDIO ARAPUCÃO\n27.812.2739.1.214\n4.4.90.51.00\n300.000,00\n300.000,00\nCONSTRUÇÃO DE PRAÇA DE ESPORTE- ST. SANTO ANTÔNIO\n27.812.2739.

In [93]:
aplicacoes = []
for m in re.finditer(r'(.+)\n(\d{2}\.\d{3}\.\d{4}\.\d\.\d{3})', text_global):
    aplicacoes.append([m.start(), m.end(), list(m.groups())])
aplicacoes.append([len(text_global),len(text_global),None])
for i in range(len(aplicacoes)-1):
    aplic = aplicacoes[i]
    x1 = aplicacoes[i][1]
    x2 = aplicacoes[i+1][0]

    natdespesas = []

    for n in re.finditer(r'(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n(\d{1,3}(?:\.\d{3})*,\d{2})', text_global[x1:x2]):
        natdespesas.append(list(n.groups()))

    aplic.append(natdespesas)
    
aplicacoes.pop()
aplicacoes = list(map(lambda x: (x[2][1], x[2][0], x[0], x[1], tuple(x[3])), aplicacoes))

In [94]:
aplicacoes

[('01.031.0139.1.001',
  'OBRAS, INSTAL.EQUIP.CAMARA MUNICIPAL',
  332,
  386,
  (['4.4.90.51.00', '1.487.025,43'], ['4.4.90.52.00', '1.070.000,00'])),
 ('01.031.0139.2.001',
  'MANUT.ATIV.DA CAMARA MUNICIPAL',
  452,
  500,
  (['3.1.90.11.00', '17.500.000,00'],
   ['3.1.90.16.00', '4.100,00'],
   ['3.1.90.96.00', '70.000,00'],
   ['3.3.90.14.00', '75.000,00'],
   ['3.3.90.30.00', '660.000,00'],
   ['3.3.90.33.00', '1.000,00'],
   ['3.3.90.34.00', '2.500,00'],
   ['3.3.90.35.00', '5.000,00'],
   ['3.3.90.36.00', '90.000,00'],
   ['3.3.90.37.00', '10.500,00'],
   ['3.3.90.39.00', '770.000,00'],
   ['3.3.90.40.00', '160.000,00'],
   ['3.3.90.92.00', '200,00'])),
 ('01.271.2839.9.061',
  'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
  814,
  872,
  (['3.1.90.13.00', '2.100.000,00'], ['3.1.90.92.00', '200,00'])),
 ('01.272.2839.9.001',
  'ENC.PREV. C/O REGIME PRÓPRIO DE PREVIDÊNCIA',
  932,
  993,
  (['3.1.91.13.00', '1.580.000,00'],
   ['3.1.91.92.00', '200,00'],
   ['3.3.90.08.00', '200,00

In [54]:
unidades_temp = []
for n in re.finditer(r'(?:Quadro de Detalhamento da Despesa\nÓRGÃO\n)(.+)\n(.+)\n(.+)\n(.+)\n(.+)\n(.+)\n(.+)\n', text_global):
    unidades_temp.append([n.start(), n.end(), list(n.groups())])
    
unidades_temp.append([len(text_global),len(text_global),None])

In [55]:
unidades_temp[0]

[97,
 221,
 ['CAMARA MUNICIPAL DE JATAI',
  'CÓDIGO',
  '01',
  'UNIDADE ORÇAMENTÁRIA',
  'CAMARA MUNICIPAL',
  'CÓDIGO',
  '01']]

In [56]:
unidades = []
for i in range(len(unidades_temp)-1):
    unidade = unidades_temp[i]
    x1 = unidades_temp[i][1]
    x2 = unidades_temp[i+1][0]
    
    unidade = [(unidade[2][2], unidade[2][0]), (unidade[2][6], unidade[2][4]), []]
    for aplic in aplicacoes:
        if aplic[2] >= x1 and aplic[3] <= x2:
            unidade[-1].append(aplic)    
    unidades.append(unidade)    

In [57]:
unidades_temp = unidades
unidades = []

orgao = ''
unidade_orc = ''
for u in unidades_temp:
    if u[0] != orgao or u[1] != unidade_orc:
        orgao = u[0]
        unidade_orc = u[1]
        unidades.append([orgao, unidade_orc, []])
    unidades[-1][-1].extend(u[2])

In [58]:
unidades[0]

[('01', 'CAMARA MUNICIPAL DE JATAI'),
 ('01', 'CAMARA MUNICIPAL'),
 [('01.031.0139.1.001',
   'OBRAS, INSTAL.EQUIP.CAMARA MUNICIPAL',
   332,
   386,
   (['4.4.90.51.00', '1.487.025,43'], ['4.4.90.52.00', '1.070.000,00'])),
  ('01.031.0139.2.001',
   'MANUT.ATIV.DA CAMARA MUNICIPAL',
   452,
   500,
   (['3.1.90.11.00', '17.500.000,00'],
    ['3.1.90.16.00', '4.100,00'],
    ['3.1.90.96.00', '70.000,00'],
    ['3.3.90.14.00', '75.000,00'],
    ['3.3.90.30.00', '660.000,00'],
    ['3.3.90.33.00', '1.000,00'],
    ['3.3.90.34.00', '2.500,00'],
    ['3.3.90.35.00', '5.000,00'],
    ['3.3.90.36.00', '90.000,00'],
    ['3.3.90.37.00', '10.500,00'],
    ['3.3.90.39.00', '770.000,00'],
    ['3.3.90.40.00', '160.000,00'],
    ['3.3.90.92.00', '200,00'])),
  ('01.271.2839.9.061',
   'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
   814,
   872,
   (['3.1.90.13.00', '2.100.000,00'], ['3.1.90.92.00', '200,00'])),
  ('01.272.2839.9.001',
   'ENC.PREV. C/O REGIME PRÓPRIO DE PREVIDÊNCIA',
   932,
   99

In [59]:
funcoes = re.split(r'\||\n', '''01|LEGISLATIVA
02|JUDICIÁRIA
03|ESSENCIAL A JUSTIÇA
04|ADMINISTRAÇÃO
05|DEFESA NACIONAL
06|SEGURANÇA PÚBLICA
07|RELAÇÕES EXTERIORES
08|ASSISTÊNCIA SOCIAL
09|PREVIDÊNCIA SOCIAL
10|SAÚDE
11|TRABALHO
12|EDUCAÇÃO
13|CULTURA
14|DIREITOS DA CIDADANIA
15|URBANISMO
16|HABITAÇÃO
17|SANEAMENTO
18|GESTÃO AMBIENTAL
19|CIÊNCIAS E TECNOLOGIA
20|AGRICULTURA
21|ORGANIZAÇÃO AGRÁRIA
22|INDÚSTRIA
23|COMÉRCIO E SERVIÇOS
24|COMUNICAÇÕES
25|ENERGIA
26|TRANSPORTE
27|DESPORTO E LAZER
28|ENCARGOS ESPECIAIS
99|RESERVA DE CONTINGÊNCIA''')

In [60]:
funcoes = [(funcoes[i], funcoes[i+1]) for i in range(0, len(funcoes)-1, 2)]

In [61]:
funcoes = sorted(funcoes, key=lambda x: x[0])

In [62]:
loa = models.Loa.objects.get(pk=2024)

In [63]:
for c, e in funcoes:
    created, f = models.Funcao.objects.get_or_create(loa=loa, codigo=c, especificacao=e)

In [64]:
models.Funcao.objects.count()

29

In [65]:
subfs = re.split(r'\t|\n', '''01	031	AÇÃO LEGISLATIVA
01	032	CONTROLE EXTERNO
02	061	AÇÃO JUDICIÁRIA
02	062	DEFESA DO INTERESSE PÚBLICO NO PROCESSO JUDICIÁRIO
03	091	DEFESA DA ORDEM JURÍDICA
03	092	REPRESENTAÇÃO JUDICIAL E EXTRAJUDICIAL
04	121	PLANEJAMENTO E ORÇAMENTO
04	122	ADMINISTRAÇÃO GERAL
04	123	ADMINISTRAÇÃO FINANCEIRA
04	124	CONTROLE INTERNO
04	125	NORMATIZAÇÃO E FISCALIZAÇÃO
04	126	TECNOLOGIA DA INFORMAÇÃO
04	127	ORDENAMENTO TERRITORIAL
04	128	FORMAÇÃO DE RECURSOS HUMANOS
04	129	ADMINISTRAÇÃO DE RECEITAS
04	130	ADMINISTRAÇÃO DE CONCESSÕES
04	131	COMUNICAÇÃO SOCIAL
05	151	DEFESA AÉREA
05	152	DEFESA NAVAL
05	153	DEFESA TERRESTRE
06	181	POLICIAMENTO
06	182	DEFESA CIVIL
06	183	INFORMAÇÃO E INTELIGÊNCIA
07	211	RELAÇÕES DIPLOMÁTICAS
07	212	COOPERAÇÃO INTERNACIONAL
08	241	ASSISTÊNCIA AO IDOSO
08	242	ASSISTÊNCIA AO PORTADOR DE DEFICIÊNCIA
08	243	ASSISTÊNCIA A CRIANÇA E  AO ADOLESCENTE
08	244	ASSISTÊNCIA COMUNITÁRIA
09	271	PREVIDÊNCIA BÁSICA
09	272	PREVIDÊNCIA DO REGIME ESTATUTÁRIO
09	273	PREVIDÊNCIA COMPLEMENTAR
09	274	PREVIDÊNCIA ESPECIAL
10	301	ATENÇÃO BÁSICA
10	302	ASSISTÊNCIA HOSPITALAR E AMBULATORIAL
10	303	SUPORTE PROFILÁTICO E TERAPÊUTICO
10	304	VIGILÂNCIA SANITÁRIA
10	305	VIGILÂNCIA EPIDEMIOLÓGICA
10	306	ALIMENTAÇÃO E NUTRIÇÃO
11	331	PROTEÇÃO E BENEFÍCIOS AO TRABALHADOR
11	332	RELAÇÕES DE TRABALHO
11	333	EMPREGABILIDADE
11	334	FOMENTO AO TRABALHO
12	361	ENSINO FUNDAMENTAL
12	362	ENSINO MÉDIO
12	363	ENSINO PROFISSIONAL
12	364	ENSINO SUPERIOR
12	365	EDUCAÇÃO INFANTIL
12	366	EDUCAÇÃO DE JOVENS E ADULTOS
12	367	EDUCAÇÃO ESPECIAL
12	368	EDUCAÇÃO BÁSICA
13	391	PATRIMÔNIO HISTÓRICO, ARTÍSTICO E ARQUEOLÓGICO
13	392	DIFUSÃO CULTURAL
14	421	CUSTÓDIA E REINTEGRAÇÃO SOCIAL
14	422	DIREITOS INDIVIDUAIS, COLETIVOS E DIFUSOS
14	423	ASSISTÊNCIA AOS POVOS INDÍGENAS
15	451	INFRA-ESTRUTURA URBANA
15	452	SERVIÇOS URBANOS
15	453	TRANSPORTES COLETIVOS URBANOS
16	481	HABITAÇÃO RURAL
16	482	HABITAÇÃO URBANA
17	511	SANEAMENTO BÁSICO RURAL
17	512	SANEAMENTO BÁSICO URBANO
18	541	PRESERVAÇÃO E CONSERVAÇÃO AMBIENTAL
18	542	CONTROLE AMBIENTAL
18	543	RECUPERAÇÃO DE ÁREAS DEGRADADAS
18	544	RECURSOS HÍDRICOS
18	545	METEOROLOGIA
19	571	DESENVOLVIMENTO CIENTÍFICO
19	572	DESENVOLVIMENTO TECNOLÓGICO E ENGENHARIA
19	573	DIFUSÃO DO CONHECIMENTO CIENTÍFICO E TECNOLÓGICO
20	601	PROMOÇÃO DA PRODUÇÃO VEGETAL
20	602	PROMOÇÃO DA PRODUÇÃO ANIMAL
20	603	DEFESA SANITÁRIA VEGETAL
20	604	DEFESA SANITÁRIA ANIMAL
20	605	ABASTECIMENTO
20	606	EXTENSÃO RURAL
20	607	IRRIGAÇÃO
20	608	PROMOÇÃO DA PRODUÇÃO AGROPECUARIA
20	609	DEFESA AGROPECAUARIA
21	631	REFORMA AGRÁRIA
21	632	COLONIZAÇÃO
22	661	PROMOÇÃO INDUSTRIAL
22	662	PRODUÇÃO INDUSTRIAL
22	663	MINERAÇÃO
22	664	PROPRIEDADE INDUSTRIAL
22	665	NORMALIZAÇÃO E QUALIDADE
23	691	PROMOÇÃO COMERCIAL
23	692	COMERCIALIZAÇÃO
23	693	COMÉRCIO EXTERIOR
23	694	SERVIÇOS FINANCEIROS
23	695	TURISMO
24	721	COMUNICAÇÕES POSTAIS
24	722	TELECOMUNICAÇÕES
25	751	CONSERVAÇÃO DE ENERGIA
25	752	ENERGIA ELÉTRICA
25	753	COMBUSTÍVEIS MINERAIS
25	754	BIOCOMBUSTÍVEIS
26	781	TRANSPORTE AÉREO
26	782	TRANSPORTE RODOVIÁRIO
26	783	TRANSPORTE FERROVIÁRIO
26	784	TRANSPORTE HIDROVIÁRIO
26	785	TRANSPORTE ESPECIAIS
27	811	DESPORTO DE RENDIMENTO
27	812	DESPORTO COMUNITÁRIO
27	813	LAZER
28	841	REFINANCIAMENTO DA DÍVIDA INTERNA
28	842	REFINANCIAMENTO DA DÍVIDA EXTERNA
28	843	SERVIÇO DA DÍVIDA INTERNA
28	844	SERVIÇO DA DÍVIDA EXTERNA
28	845	OUTRAS TRANSFERÊNCIAS
28	846	OUTROS ENCARGOS ESPECIAIS
28	847	TRANSFERÊNCIAS PARA A EDUCAÇÃO BÁSICA
99	999	RESERVA DE CONTINGÊNCIA''')

In [66]:
subfs = [(subfs[i], subfs[i+1], subfs[i+2]) for i in range(0, len(subfs)-2, 3)]

In [67]:
subfs = sorted(subfs, key=lambda x: (x[0], x[1]))

In [68]:
for fc, c, e in subfs:
    f = models.Funcao.objects.filter(loa=loa, codigo=fc).first()
    created, sf = models.SubFuncao.objects.get_or_create(loa=loa, funcao=f, codigo=c, especificacao=e)

In [69]:
models.SubFuncao.objects.count()

114

In [70]:
orgao, unidade, despesas = unidades[0]

In [71]:
orgao

('01', 'CAMARA MUNICIPAL DE JATAI')

In [72]:
unidade

('01', 'CAMARA MUNICIPAL')

In [73]:
desp, espec, x1, x2, naturezas = despesas[0]

In [74]:
count = [0,0,0]
for orgao, unidade, despesas in unidades:
    count[0] += 1
    for desp, espec, x1, x2, naturezas in despesas:
        count[1] += 1
        for nat in naturezas:
            count[2] += 1
print(count)

[32, 314, 1131]


In [75]:
count = 0
for orgao, unidade, despesas in unidades:
    o_obj, created = models.Orgao.objects.get_or_create(
        loa=loa, codigo=orgao[0], especificacao=orgao[1]
    )

    uo_obj, created = models.UnidadeOrcamentaria.objects.get_or_create(
        loa=loa, orgao=o_obj, codigo=unidade[0], especificacao=unidade[1]
    )
    
    for desp, espec, x1, x2, naturezas in despesas:
        desp_re = re.match(r'(?P<funcao>\d{2})\.(?P<subfuncao>\d{3})'
                           r'\.(?P<programa>\d{4})\.(?P<acao>\d\.\d{3})', desp)

        funcao_obj = models.Funcao.objects.filter(loa=loa, codigo=desp_re['funcao']).first()
        subfunc_obj = models.SubFuncao.objects.filter(loa=loa
            codigo=desp_re['subfuncao']).first()
        
        programa_obj, created = models.Programa.objects.get_or_create(
            loa=loa, codigo=desp_re['programa'], especificacao='')
        
        acao_obj, created = models.Acao.objects.get_or_create(
            loa=loa, codigo=desp_re['acao'], especificacao=espec)
        
        
        for nat in naturezas:
            count += 1
            natureza, created = models.Natureza.objects.get_or_create(
            loa=loa, codigo=nat[0])
            
            desp_obj, created = models.Despesa.objects.get_or_create(
            loa = loa,
            orgao = o_obj,
            unidade = uo_obj,
            funcao = funcao_obj,
            subfuncao = subfunc_obj,
            programa = programa_obj,
            acao = acao_obj,
            natureza = natureza)
            
            desp_obj.valor_norma = Decimal(nat[1].replace('.', '').replace(',', '.'))
            
            try: 
                desp_obj.save()
            except Exception as e:
                print(e)
                break
        if not desp_obj.pk:
            break
            
            

print(count, models.Despesa.objects.filter(loa=loa).count())

1131 1174


In [107]:
desp = Despesa.objects.filter(loa=loa, acao__codigo='2.102', natureza__codigo='3.3.90.30.00').first()
desp.valor_norma = Decimal('202857.51')
desp.save()
desp = Despesa.objects.filter(loa=loa, acao__codigo='2.015', natureza__codigo='3.3.90.36.00').first()
desp.valor_norma = Decimal('116778.03')
desp.save()

In [76]:
view = '''run_sql(
               drop view if exists loa_despesa_consulta;
                create or replace view loa_despesa_consulta as
                    select
                        d.id as id,
                        d.loa_id as loa_id,

                        f.codigo || '.' || sf.codigo || '.' || p.codigo || '.' || a.codigo as codigo,
                        a.especificacao as especificacao,

                        o.codigo as cod_orgao,
                        o.especificacao as esp_orgao,

                        u.codigo as cod_unidade,
                        u.especificacao as esp_unidade,

                        n.codigo as cod_natureza,

                        fte.codigo as cod_fonte


                    from loa_despesa d
                        inner join loa_loa        l on (d.loa_id = l.id)

                        inner join loa_funcao     f on (d.funcao_id = f.id)
                        inner join loa_subfuncao sf on (d.subfuncao_id = sf.id)

                        inner join loa_programa   p on (d.programa_id = p.id)
                        inner join loa_acao       a on (d.acao_id = a.id)

                        inner join loa_natureza   n on (d.natureza_id = n.id)

                        left join loa_fonte     fte on (d.fonte_id = fte.id)

                        inner join loa_orgao                  o on (d.orgao_id = o.id)
                        inner join loa_unidadeorcamentaria    u on (d.unidade_id = u.id)

;)
result = run_sql('select * from loa_despesa_consulta')
for r in result[:30]:
    print(r)'''

In [77]:
"""models.Despesa.objects.all().delete()
models.Orgao.objects.all().delete()
models.UnidadeOrcamentaria.objects.all().delete()
models.Funcao.objects.all().delete()
models.SubFuncao.objects.all().delete()
models.Programa.objects.all().delete()
models.Acao.objects.all().delete()
models.Natureza.objects.all().delete()
models.EmendaLoaRegistroContabil.objects.all().delete()"""

'models.Despesa.objects.all().delete()\nmodels.Orgao.objects.all().delete()\nmodels.UnidadeOrcamentaria.objects.all().delete()\nmodels.Funcao.objects.all().delete()\nmodels.SubFuncao.objects.all().delete()\nmodels.Programa.objects.all().delete()\nmodels.Acao.objects.all().delete()\nmodels.Natureza.objects.all().delete()\nmodels.EmendaLoaRegistroContabil.objects.all().delete()'

In [78]:
text_global = load_pages_with(page_mask = 'ADENDO III A PORTARIA SOF N.08 DE 04/02/85')

In [79]:
text_global[:500]

'\nESTADO DE GOIÁS\nEXERCÍCIO DE 2024\nMUNICÍPIO DE JATAI\nPÁGINA\n1\nRUA ITARUMA, 355\n- S. SANTA MARIA\nLei 4.320/ 64\n- Anexo\nIl\nADENDO III A PORTARIA SOF N.08 DE 04/02/85\nORCAMENTO PROGRAMA - ADMINISTRACAO DIRETA E INDIRETA\nÓRGÃO\n01\n- CAMARA MUNICIPAL DE JATAI\nUNIDADE ORÇAMENTÁRIA\n01\n- CAMARA MUNICIPAL\nCÓDIGO\nESPECIFICAÇÕES\nELEMENTO\nNATUREZA\nCAT.ECONÔMICA\n3.0.00.00.00 | DESPESAS CORRENTES\n23.292.974,57\n3.1.00.00.00 | PESSOAL\n21.518.074,57\n3.1.90.11.00 | Vencimentos e Vantagens Fixas - Pessoal Civil\n17'

In [80]:
naturezas = []
for m in re.finditer(r'\n(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2}) \| (.+)\n', text_global):
    naturezas.append(m.groups())

In [81]:
naturezas[:3]

[('3.0.00.00.00', 'DESPESAS CORRENTES'),
 ('3.1.00.00.00', 'PESSOAL'),
 ('3.1.90.11.00', 'Vencimentos e Vantagens Fixas - Pessoal Civil')]

In [82]:
for cod, esp in naturezas:
    models.Natureza.objects.filter(loa=loa, codigo=cod).update(especificacao=esp)

In [1]:
text_global = load_pages_with(page_mask='ADENDO V A PORTARIA SOF N.15 DE 20/06/78')

NameError: name 'load_pages_with' is not defined

In [ ]:
programas = set()
for m in re.finditer(r'\n(\d{4})\n(.+)\n', text_global):
    programas.add(m.groups())

In [ ]:
programas

In [ ]:
for cod, esp in programas:
    Programa.objects.filter(loa=loa,codigo=cod).update(especificacao=esp)